# Combining Data for Final Database

In [2]:
import pandas as pd
import json
import os
import numpy as np

In [3]:
DATA_FOLDER = "../data/preprocessed_sustainability_reports/"

In [23]:
[file for file in os.listdir(DATA_FOLDER)]

['all_asset_managers_preprocessed_vfinal_BERT.json',
 '.DS_Store',
 'all_pension_funds_preprocessed_vfinal_BERT.json',
 'all_asian_banks_preprocessed_vfinal_BERT.json',
 'all_pension_funds_preprocessed_vfinal.json',
 'all_vfinal_BERT.json',
 'all_asset_managers_preprocessed_vfinal.json',
 'all_insurance_preprocessed_vfinal_BERT.json',
 'all_vfinal.json',
 'all_asian_banks_preprocessed_vfinal.json',
 'all_insurance_preprocessed_vfinal.json']

### Combine all 4 json into 1

In [16]:
files = [file for file in os.listdir(DATA_FOLDER) if "BERT" not in file and ".DS" not in file]
files_BERT = [file for file in os.listdir(DATA_FOLDER) if "BERT" in file]

def merge_JsonFiles(filename):
    result = list()
    for f1 in filename:
        with open(DATA_FOLDER + f1, 'r') as infile:
            result.extend(json.load(infile))

    with open(DATA_FOLDER +'all_vfinal.json', 'w') as output_file:
        json.dump(result, output_file)

merge_JsonFiles(files)

In [26]:
def merge_JsonFiles(filename):
    result = list()
    for f1 in filename:
        with open(DATA_FOLDER + f1, 'r') as infile:
            result.extend(json.load(infile))

    with open(DATA_FOLDER + 'all_vfinal_BERT.json', 'w') as output_file:
        json.dump(result, output_file)

merge_JsonFiles(files_BERT)

In [130]:
with open(DATA_FOLDER + 'all_vfinal_BERT.json', 'r') as infile:
    all_data = json.load(infile)

In [131]:
len(all_data)

236

### Combine text data predictions with charts

In [120]:
DATA_FOLDER = "../data/dashboard_data/"
with open(DATA_FOLDER + 'all_text_output.json','r') as infile:  
    text_predictions = json.load(infile)

with open(DATA_FOLDER + 'all_chart_output.json','r') as infile:  
    chart_predictions = json.load(infile)
        

In [ ]:
text_charts = []
count = 0
count_a = 0
for company in text_predictions:
    company_t = company["company"]
    year_t = company["year"]
    flag = 0
    for company_charts in chart_predictions:
        company_c = company_charts["company"]
        year_c = company_charts["year"]
        
        if company_t == company_c and year_t == year_c:
            charts_c = company_charts["images_path"]
            company["charts_images"] = charts_c
            text_charts.append(company)
            flag = 1
            count+=1
            break
           
    if flag == 0 :
        count_a +=1
        company["charts_images"] = np.nan 
        text_charts.append(company)

In [122]:
len(text_charts)

236

In [123]:
with open(DATA_FOLDER +'all_text_chart_output.json', 'w') as output_file:
    json.dump(text_charts, output_file)


### Combine text & chart data with tables & wordcloud

In [230]:
DATA_FOLDER = "../data/dashboard_data/"
with open(DATA_FOLDER + 'all_text_chart_wordcloud.json','r') as infile:  
    all_except_table = json.load(infile)

In [231]:
len(all_except_table)

236

In [232]:
with open(DATA_FOLDER + 'all_tables.json','r') as infile:  
    all_tables = json.load(infile)

In [233]:
len(all_tables)

230

In [234]:
final_database = []
count = 0
count_a = 0
for company in all_except_table:
    company_t = company["company"]
    year_t = company["year"]
    flag = 0
    for table in all_tables:
        company_c = table["company"]
        year_c = table["year"]
        
        if company_t == company_c and year_t == year_c:
            company["table_keywords"] = table["table_keywords"]
            company["table_image_keywords"] = table["table_image_keywords"]
            company["table_images"]    = table["table_images"]           
            final_database.append(company)
            flag = 1
            count+=1
            break
           
    if flag == 0 :
        count_a +=1
        company["table_keyowrds"] = "nan"
        company["table_image_keyowrds"] = "nan" 
        company["table_images"] = "nan"    
        final_database.append(company)

In [235]:
final_database_v1 = []
for fi in final_database:
    chart_images = fi["charts_images"]
   
    if chart_images == "nan":
        final_database_v1.append(fi)
        continue
    
    elif chart_images == {}:
        fi["charts_images"] = "nan"
        final_database_v1.append(fi)
        continue
        
    for page, images in chart_images.items():
        image_new = []
        if len(images) != 0 :
            for image in images:
                new_path = "data/dashboard_data/"+ image
                image_new.append(new_path)
        fi["charts_images"][page] = image_new
    final_database_v1.append(fi)
                

In [236]:
with open(DATA_FOLDER +'final_database.json', 'w') as output_file:
    json.dump(final_database_v1, output_file)


In [5]:
DATA_FOLDER = "../data/dashboard_data/"
with open(DATA_FOLDER + 'final_database.json','r') as infile:  
    alls = json.load(infile)

In [8]:
count = 0
for fi in alls:
    text_output = fi["text_output"]
    pages = text_output["page"]
    if len(pages) == 0 :
        count+=1

In [9]:
count

24

In [ ]:
# throw error if PDF cannot get
# throw error if text_output is nothing